In [35]:
import numpy as np
import pandas as pd
import time, random

In [37]:
df_GEI = pd.read_csv(r"/database/GEI_scraped.csv")
df_metrics = pd.read_csv(r"/database/Ei_metrics_beforeGEI.csv", dtype={"GAME_ID": str})

In [38]:
def match_gei(row):
    matched = df_GEI[
        (df_GEI["Date"] == row["DATE"]) &
        (df_GEI["Score"].str.contains(row["TEAM1"], case=False, na=False)) &
        (df_GEI["Score"].str.contains(row["TEAM2"], case=False, na=False))
    ]
    return matched["Excitment"].mean() if not matched.empty else 0


In [39]:
df_metrics["GEI"] = df_metrics.apply(match_gei, axis=1)

In [40]:
df_metrics.head(20)

,GAME_ID,TEAM1,TEAM2,DATE,PF,3PM,DUNKS,OT,CLUTCH,GEI
0,0022401186,ATL,ORL,2025-04-13,23,32,16,0,0,4.9
1,0022401188,BKN,NYK,2025-04-13,49,28,5,0,1,7.5
2,0022401195,UTA,MIN,2025-04-13,31,29,11,0,0,5.1
3,0022401194,MEM,DAL,2025-04-13,28,20,19,0,0,3.2
4,0022401196,OKC,NOP,2025-04-13,44,16,6,0,0,3.7
5,0022401200,SAC,PHX,2025-04-13,16,29,7,0,0,5.0
6,0022401192,MIL,DET,2025-04-13,36,45,5,1,1,9.1
7,0022401187,BOS,CHA,2025-04-13,30,25,4,0,1,0.0
8,0022401199,POR,LAL,2025-04-13,37,24,12,0,0,3.5
9,0022401193,DEN,HOU,2025-04-13,37,22,13,0,0,3.2


In [41]:
df_metrics.loc[df_metrics['GEI']==0, 'GEI']=df_metrics['GEI'].mean()
df_metrics[df_metrics['GEI']==0]

,GAME_ID,TEAM1,TEAM2,DATE,PF,3PM,DUNKS,OT,CLUTCH,GEI


In [53]:
df_metrics.to_csv("Ei_metrics_beforePerf.csv", index=False)

In [2]:
from nba_api.stats.endpoints import BoxScoreTraditionalV2

In [30]:
player_stat_data = BoxScoreTraditionalV2(game_id='0022401161')
stats_df = player_stat_data.player_stats.get_data_frame()
stats_df.head(29)


,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS


In [23]:
bs=stats_df
df=bs[(bs['PTS']>=40) | (bs['BLK']>= 7) | (bs['STL']>=6) | (bs['AST']>= 18) | (bs['REB']>= 21) | ((bs['PTS']>=30) & (bs['AST']>= 10)) | ((bs['PTS']>=30) & (bs['REB']>= 15)) | ((bs['PTS']>=20) & (bs['AST']>= 10) & (bs['REB']>= 10))]
len(df)

2

In [ ]:
perf_dict={}
for game in game_ids:

    data = BoxScoreTraditionalV2(game_id=game, timeout=100)
    bs = data.player_stats.get_data_frame()

    df=bs[(bs['PTS']>=40) | (bs['BLK']>= 7) | (bs['STL']>=6) | (bs['AST']>= 18) | (bs['REB']>= 21) | ((bs['PTS']>=30) & (bs['AST']>= 10)) | ((bs['PTS']>=30) & (bs['REB']>= 15)) | ((bs['PTS']>=20) & (bs['AST']>= 10) & (bs['REB']>= 10))]
    perf_dict[game]= len(df)

                # Respectful delay to avoid rate limiting
    time.sleep(random.uniform(0.8, 1.5))

len(perf_dict)
data = pd.DataFrame(list(perf_dict.items()), columns=['GAME_ID', 'COUNT'])
data.to_csv("performances_count.csv", index= False)

In [42]:
df_perf = pd.read_csv(r"/database/performances_count.csv", dtype={"GAME_ID": str})

In [61]:
def match_perf(row):
    matched= df_perf[df_perf["GAME_ID"]==row["GAME_ID"]]
    return int(matched["COUNT"].mean()) if not matched.empty else 0

df_metrics["PERFORMANCES"]= df_metrics.apply(match_perf, axis=1)

In [63]:
df_metrics.to_csv("Ei_metrics.csv", index=False)

In [65]:
#getting final EI TABLE (post game data)

In [79]:
metrics = pd.read_csv(r"/database/Ei_metrics.csv", dtype={"GAME_ID": str})

In [80]:
metrics.drop(['PERFOMANCES'], axis=1, inplace=True)
metrics

,GAME_ID,TEAM1,TEAM2,DATE,PF,3PM,DUNKS,OT,CLUTCH,GEI,PERFORMANCES
0,0022401186,ATL,ORL,2025-04-13,23,32,16,0,0,4.9,0
1,0022401188,BKN,NYK,2025-04-13,49,28,5,0,1,7.5,0
2,0022401195,UTA,MIN,2025-04-13,31,29,11,0,0,5.1,1
3,0022401194,MEM,DAL,2025-04-13,28,20,19,0,0,3.2,0
4,0022401196,OKC,NOP,2025-04-13,44,16,6,0,0,3.7,0
...,...,...,...,...,...,...,...,...,...,...,...
1225,0022400070,MEM,UTA,2024-10-23,59,27,12,0,1,9.1,0
1226,0022400067,CLE,TOR,2024-10-23,46,26,13,0,0,3.1,0
1227,0022400069,CHI,NOP,2024-10-23,33,24,11,0,0,5.8,0
1228,0022400061,BOS,NYK,2024-10-22,27,40,4,0,0,2.0,1


In [81]:
metrics["norm_GEI"]= (metrics["GEI"] - metrics["GEI"].min())/(metrics["GEI"].max() - metrics["GEI"].min())

metrics["norm_PERFORMANCES"]= (metrics["PERFORMANCES"] - metrics["PERFORMANCES"].min())/(metrics["PERFORMANCES"].max() - metrics["PERFORMANCES"].min())

metrics["norm_DUNKS"]= (metrics["DUNKS"] - metrics["DUNKS"].min())/(metrics["DUNKS"].max() - metrics["DUNKS"].min())

metrics["norm_3PM"]= (metrics["3PM"] - metrics["3PM"].min())/(metrics["3PM"].max() - metrics["3PM"].min())

metrics["norm_PF"]= (metrics["PF"] - metrics["PF"].min())/(metrics["PF"].max() - metrics["PF"].min())

metrics["norm_CLUTCH"]= metrics["CLUTCH"]

metrics["norm_OT"] = np.where(metrics["OT"] == 1, 0.75,
                      np.where(metrics["OT"] == 2, 0.95,
                      np.where(metrics["OT"] >= 3, 1,0)))


metrics["EI"] = ((0.30 * metrics["norm_GEI"]) + (0.20 * metrics["norm_PERFORMANCES"]) + (0.15 * metrics["norm_CLUTCH"]) + (0.15 * metrics["norm_OT"]) + (0.1 * metrics["norm_3PM"]) + (0.1 * metrics["norm_DUNKS"])) - (0.05 * metrics["norm_PF"])

In [85]:
metrics

,GAME_ID,TEAM1,TEAM2,DATE,PF,3PM,DUNKS,OT,CLUTCH,GEI,PERFORMANCES,norm_GEI,norm_PERFORMANCES,norm_DUNKS,norm_3PM,norm_PF,norm_CLUTCH,norm_OT,EI
0,0022401186,ATL,ORL,2025-04-13,23,32,16,0,0,4.9,0,0.260870,0.000000,0.652174,0.529412,0.14,0,0.0,0.189419
1,0022401188,BKN,NYK,2025-04-13,49,28,5,0,1,7.5,0,0.449275,0.000000,0.173913,0.411765,0.66,1,0.0,0.310350
2,0022401195,UTA,MIN,2025-04-13,31,29,11,0,0,5.1,1,0.275362,0.333333,0.434783,0.441176,0.30,0,0.0,0.221871
3,0022401194,MEM,DAL,2025-04-13,28,20,19,0,0,3.2,0,0.137681,0.000000,0.782609,0.176471,0.24,0,0.0,0.125212
4,0022401196,OKC,NOP,2025-04-13,44,16,6,0,0,3.7,0,0.173913,0.000000,0.217391,0.058824,0.56,0,0.0,0.051795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,0022400070,MEM,UTA,2024-10-23,59,27,12,0,1,9.1,0,0.565217,0.000000,0.478261,0.382353,0.86,1,0.0,0.362627
1226,0022400067,CLE,TOR,2024-10-23,46,26,13,0,0,3.1,0,0.130435,0.000000,0.521739,0.352941,0.60,0,0.0,0.096598
1227,0022400069,CHI,NOP,2024-10-23,33,24,11,0,0,5.8,0,0.326087,0.000000,0.434783,0.294118,0.34,0,0.0,0.153716
1228,0022400061,BOS,NYK,2024-10-22,27,40,4,0,0,2.0,1,0.050725,0.333333,0.130435,0.764706,0.22,0,0.0,0.160398


In [84]:
metrics.to_csv("2024_postgame.csv", index=False)